In [34]:
import numpy as np
from tqdm.notebook import tqdm
import torch
import gpytorch
import botorch
from pak.function import *
from pak.GPmodel import fit_gp_model
from pak.plotfunc import visualize_2d_contour
from skopt.sampler import Lhs
import warnings
warnings.filterwarnings("ignore")

torch.set_default_tensor_type(torch.DoubleTensor)


In [35]:
class GPModel(gpytorch.models.ExactGP, botorch.models.gpytorch.GPyTorchModel):
    _num_outputs = 1

    def __init__(self, train_x, train_y, likelihood):
        super().__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


def fit_gp_model(train_x, train_y, num_train_iters=500):
    # declare the GP
    noise = 1e-4

    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPModel(train_x, train_y, likelihood)
    model.likelihood.noise = noise

    # train the hyperparameter (the constant)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    model.train()
    likelihood.train()

    for i in range(num_train_iters):
        optimizer.zero_grad()

        output = model(train_x)
        loss = -mll(output, train_y)

        loss.backward()
        optimizer.step()

    model.eval()
    likelihood.eval()

    return model, likelihood

In [36]:
f = F1

In [37]:
bound = 5

train_x = torch.tensor([[1.0,2.0], [2.0,3.0]])
train_y = torch.tensor(-f(train_x.numpy()))

model, likelihood = fit_gp_model(train_x, train_y)

In [ ]:
train_y

In [ ]:
num_queries = 10

for i in range(num_queries):
    print("iteration", i)
    print("incumbent", train_x[train_y.argmax()], train_y.max())

    model, likelihood = fit_gp_model(train_x, train_y)

    policy = botorch.acquisition.analytic.ExpectedImprovement(
        model, best_f=train_y.max()
    )

    next_x, acq_val = botorch.optim.optimize_acqf(
        policy,
        bounds=torch.tensor([[-bound * 1.0,-bound * 1.0], [bound * 1.0,bound * 1.0]]),
        q=1,
        num_restarts=20,
        raw_samples=50,
    )


    next_y = torch.tensor(-f(next_x.numpy()))

    train_x = torch.cat([train_x, next_x])
    train_y = torch.cat([train_y, next_y])